# Treinamento Multilayer Perceptron

**Python:** 3.12.2

## Imports

In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import dependencies as dep

In [2]:
PATH = '../data/data.csv'
PATH_OVERSAMPLED = '../data/data_balanced.csv'
MODEL_PATH = '../xgboost.pkl'

X = 1
O = -1
BLANK = 0

O_WIN   = 0
DRAW    = 1
ONGOING = 2
X_WIN   = 3
class_names = ['O_WIN', 'DRAW', 'ONGOING', 'X_WIN']

## Preparo dos Dados

In [3]:
df = pd.read_csv(PATH)

feature_cols = [str(i) for i in range(9)]
X = df[feature_cols]
y = df['category']

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

## Treinamento

In [5]:
# Criação e treino do modelo XGBoost
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',  # função de avaliação para múltiplas classes
    random_state=42
)
xgb.fit(X_train, y_train)

c:\Users\Marnie\anaconda3\envs\TicTacToe\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:31:29] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [22]:
# Avaliação no conjunto de validação
y_pred_val = xgb.predict(X_val)
print("Validation Accuracy :", accuracy_score(y_val, y_pred_val))
print("Validation Precision:", precision_score(y_val, y_pred_val, average='weighted'))
print("Validation Recall   :", recall_score(y_val, y_pred_val, average='weighted'))
print("Validation F1-score :", f1_score(y_val, y_pred_val, average='weighted'))

print("\nClassification Report (Validation):\n",
      classification_report(y_val, y_pred_val, digits=4))

Validation Accuracy : 0.9826589595375722
Validation Precision: 0.9814867839438788
Validation Recall   : 0.9826589595375722
Validation F1-score : 0.9820243269183538

Classification Report (Validation):
               precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       188
           1     0.4000    0.3333    0.3636         6
           2     0.9706    0.9635    0.9670       137
           3     0.9895    1.0000    0.9947       188

    accuracy                         0.9827       519
   macro avg     0.8400    0.8242    0.8313       519
weighted avg     0.9815    0.9827    0.9820       519



## Teste

In [ ]:
# Avaliação no conjunto de teste
y_pred_test = xgb.predict(X_test)
dep.test_model(xgb, X_test, y_test, class_names)

NameError: name 'xgb' is not defined

In [24]:
# Salvando o modelo
joblib.dump(xgb, MODEL_PATH)
print(f'Modelo salvo em {MODEL_PATH}')

Modelo salvo em ../xgboost.pkl
